# Análisis de mensajes en archivo de texto

## Importamos las librerías a usar en el proyecto

In [292]:
import re
import pandas as pd
import numpy as np
import emoji
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

## Transformar texto a dataframe

### Creamos funciones para procesar el texto

In [293]:
def date_time(s):
    """Buscar fecha y hora en formato d/m/aaaa hh:mm"""
    pattern =  '^([0-9]+)/([0-9]+)/([0-9]+) [0-9]+:[0-9]+ -'    # El patrón corresponde a "dd/mm/aaaa hh:mm"
    result = re.match(pattern, s)                                # Busca un match entre el patrón y el string
    if result:                                                   # Si se da el match entonces la función devolverá un True,
        return True                                              # lo usaremos luego como un filtro para continuar el proceso
    return False                                                 # de trasnformación de texto a dataframe

In [294]:
def find_author(s):
    """Buscar remitente del mensaje"""
    s = s.split(":",1)              # Se divide el string a partir del primer ": "
    if len(s)==2:                   # Comprueba si el resultado de la segmentación arroja dos partes ["parte 1": "parte 2"]
        return True                 # También se usa como un filtro en el proceso de transformación de texto a dataframe
    else:
        return False

In [295]:
def getDataPoint(line):
    """Obtener los elementos que conforman un registro (Fecha, Hora, Remitente, Mensaje)"""
    
    splitline = line.split(' - ',1)             # Divide el string a partir del primer " - "
    dateTime = splitline[0]                     # Con la primer parte crea la variable de fecha y hora
    date, time = dateTime.split(" ")            # Divide al string de fecha y hora según " " y crea una variable para cada parte
    message = " ".join(splitline[1:])           # Une todas las partes del string inicial a partir de la segunda y crea la 
                                                # variable del mensaje
    if find_author(message):                    # Si la función find_author() es verdadera entonces
        splitmessage = message.split(": ",1)    # toma la variable de mensaje y divide ese string según ": "
        author = splitmessage[0]                # Con la primer parte crea la variable del remitente
        message = " ".join(splitmessage[1:])    # Une desde la segunda parte en adelante usando " " y reemplaza
                                                # la variable inicial del mensaje
    else:                                       # Si la función find_author() no es verdadera entonces
        author = None                           # Crea una variable de autor None
    return date, time, author, message          # Devuelve las variables de fecha, hora, autor y mensaje

### Importar archivo y procesar datos para usar con pandas

In [296]:
conversation = 'chat2.txt'                                                  # Creamos una variable con la ruta del archivo de texto

data = []                                                                  # Creamos una lista vacía donde se agregarán listas de variables fecha, tiempo, autor

with open(conversation, encoding='utf-8') as chat:                         # Abrimos el archivo y lo nombramos chat
    chat.readline()                                                        # Leer la primer línea del archivo (así ya no la vuelve a leer en el proceso de transformación)
    messageBuffer = []                                                     # Creamos una lista vacía donde se unirán las variables antes de guardarlas
    date, time, author = None, None, None                                  # creamos las variables de fecha, hora y autor con valor None para cada una
    
    while True:                                                            # Iterar cada vez que se arroje un True
        line = chat.readline()                                             # Creamos una variable para leer una línea de chat a la vez
        if not line:                                                       # Si la línea está vacía entonces
            break                                                          # romper el bucle y pasar a la siguiente línea
        
        # Cuando la línea no está vacía
        line = line.strip()                                                # Eliminar espacios vacíos al incio y final de la línea
        if date_time(line):                                                # Si la función date_time() arroja True entonces
            if len(messageBuffer) > 0:                                     # Si la variable messageBuffer tiene uno o más elementos entonces
                data.append([date, time, author, ' '.join(messageBuffer)]) # Agregar a la variable data una nueva lista con los valores de las variables de fecha, tiempo y autor separados por un " "

            else:
                messageBuffer.clear()                                          # Si la variable messageBuffer está vacía, limpiar la variable messageBuffer
                date, time, author, message = getDataPoint(line)               # A las variables fecha, tiempo, autor y mensaje asignarle los valores que arroja la función getDataPoint()
                messageBuffer.append(message)                                  # A la variable messageBuffer agregarle la variable del mensaje
        else:                                                              # Si la función date_time() no arroja True entonces
            messageBuffer.append(line)                                     # A la variable messageBuffer agregarle el valor de la variable de línea

### Crear dataframe

In [300]:
df = pd.DataFrame(data, columns=['Date', 'Time', 'Author', 'Message'])
df['Date'] = pd.to_datetime(df['Date'], format = "%d/%m/%Y")
df

,Date,Time,Author,Message
0,2019-09-06,11:06,Julieta Lara AIESEC Salta,Si mi nombre es Julieta Belén Lara


In [298]:
data

[['6/9/2019',
  '11:06',
  'Julieta Lara AIESEC Salta',
  'Si mi nombre es Julieta Belén Lara']]

In [299]:
chat = open(conversation, encoding='utf-8')
chat.readline()
getDataPoint(chat.readline())[3]

'Si mi nombre es Julieta Belén Lara\n'